Implementation of CNN using PyTorch Tutorial for Image classification.
1. Load and normalize any dataset available in PyTorch
2. Design LeNet model using PyTorch.
3. Train and Test the model on the selected dataset
4. Experiment with different number of filters, kernel sizes, number of layers.
5. Compare the results.

Reference Link:
[Tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

In [9]:
# importing the required libraries
# This Assignment is done by CS20B1044 Avinash R Changrani
import torch
import torchvision
import torchvision.transforms as transforms

# Load and normalize the dataset
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Define the classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [10]:
# Designing LeNet Model using Pytorch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self, num_filters, kernel_size, num_layers):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, num_filters, kernel_size=kernel_size)
        self.conv2 = nn.Conv2d(num_filters, num_filters, kernel_size=kernel_size)
        self.fc1 = nn.Linear(num_filters * 5 * 5, num_layers)
        self.fc2 = nn.Linear(num_layers, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [11]:
# Training and Testing the model
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet(num_filters=32, kernel_size=5, num_layers=128).to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# Test the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on test set: %d %%' % (100 * correct / total))

[1,  2000] loss: 1.972
[1,  4000] loss: 1.604
[1,  6000] loss: 1.493
[1,  8000] loss: 1.400
[1, 10000] loss: 1.351
[1, 12000] loss: 1.273
[2,  2000] loss: 1.186
[2,  4000] loss: 1.185
[2,  6000] loss: 1.141
[2,  8000] loss: 1.129
[2, 10000] loss: 1.095
[2, 12000] loss: 1.072
[3,  2000] loss: 0.978
[3,  4000] loss: 0.956
[3,  6000] loss: 0.954
[3,  8000] loss: 0.985
[3, 10000] loss: 0.959
[3, 12000] loss: 0.940
[4,  2000] loss: 0.831
[4,  4000] loss: 0.851
[4,  6000] loss: 0.863
[4,  8000] loss: 0.840
[4, 10000] loss: 0.840
[4, 12000] loss: 0.850
[5,  2000] loss: 0.709
[5,  4000] loss: 0.746
[5,  6000] loss: 0.771
[5,  8000] loss: 0.763
[5, 10000] loss: 0.758
[5, 12000] loss: 0.793
[6,  2000] loss: 0.649
[6,  4000] loss: 0.661
[6,  6000] loss: 0.678
[6,  8000] loss: 0.712
[6, 10000] loss: 0.694
[6, 12000] loss: 0.722
[7,  2000] loss: 0.584
[7,  4000] loss: 0.612
[7,  6000] loss: 0.616
[7,  8000] loss: 0.629
[7, 10000] loss: 0.639
[7, 12000] loss: 0.677
[8,  2000] loss: 0.519
[8,  4000] 

In [ ]:
# Experiment with different number of filters, kernel sizes, number of layers.
# Compare the results.

import matplotlib.pyplot as plt

# Set the parameters to be tested
num_filters_list = [8, 16, 32, 64]
kernel_size_list = [4, 5, 7]
num_layers_list = [64, 128, 256]

# Initialize a dictionary to store the results
results = {}

# Loop over the parameter combinations
for num_filters in num_filters_list:
    for kernel_size in kernel_size_list:
        for num_layers in num_layers_list:
            # Create the model with the current parameters
            model = LeNet(num_filters=num_filters, kernel_size=kernel_size, num_layers=num_layers).to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
            # Train and test the model
            for epoch in range(10):  # loop over the dataset multiple times
                running_loss = 0.0
                for i, data in enumerate(trainloader, 0):
                    # get the inputs; data is a list of [inputs, labels]
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    # zero the parameter gradients
                    optimizer.zero_grad()
                    # forward + backward + optimize
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
            correct = 0
            total = 0
            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            # Store the accuracy in the results dictionary
            results[(num_filters, kernel_size, num_layers)] = 100 * correct / total

# Plot the results
plt.figure()
for parameters, accuracy in results.items():
    plt.scatter(parameters, accuracy)
plt.xlabel('parameters')
plt.ylabel('Accuracy')
plt.show()